In [1]:
import geopandas as gpd
import os
import requests
import json
import time

from batid.services.guess_bdg_new import Guesser, PartialRoofHandler


source_url = "https://data.grandlyon.com/geoserver/metropole-de-lyon/ows?SERVICE=WFS&VERSION=2.0.0&request=GetFeature&typename=metropole-de-lyon:fpc_fond_plan_communaut.fpctoit_2018&outputFormat=application/json&SRSNAME=EPSG:4326&sortBy=gid"
source_path = './source/all.json'
workfile_path = './guess.json'

In [2]:
# First download if the file is not on the disk
if not os.path.exists(source_path):
    
    print('downloading ...')
    resp = requests.get(source_url)
    print('downloade finished')
    
    if resp.status_code == 200:
        with open(source_path, "w") as f:
            print('saving data to local file ...')
            json.dump(resp.json(), f)
            print("save finished")
    
    

In [3]:
# Convert to guess file
with open(source_path, 'r') as f:
    
    inputs = []
    
    source = json.load(f)
    
    c = 0
    for row in source['features']:
        
        inputs.append({
            'ext_id': row['id'],
            'polygon': row['geometry']
        })
    
guesser = Guesser()
guesser.create_work_file(inputs, workfile_path)

save_work_file - duration: 43.256402353999874 seconds
create_work_file - duration: 46.71256423000159 seconds


In [ ]:
# Launch the guess work with specific handler
guesser = Guesser()
guesser.handlers = [PartialRoofHandler()]

start = time.perf_counter()
guesser.guess_work_file(workfile_path)
end = time.perf_counter()


print(f"Worked on {len(guesser.guesses)} guesses")
print(f"Duration : {end - start} seconds")

- loading work file
load_work_file - duration: 32.33137863899901 seconds
- converting to batches
----- new batch
Processing 20000 guesses more. Total processed so far : 0/545451


GEOS_NOTICE: Self-intersection at or near point 4.9310221495217723 45.694166818023987 210.10836104132241

GEOS_NOTICE: Self-intersection at or near point 4.9310221495217723 45.694166818023987 210.10836104132241



guess_batch - duration: 354.83780745200056 seconds
save_work_file - duration: 43.7178621460007 seconds
----- new batch
Processing 20000 guesses more. Total processed so far : 20000/545451


GEOS_NOTICE: Self-intersection at or near point 4.8379010553359683 45.763998043952569 0

GEOS_NOTICE: Self-intersection at or near point 4.8379010553359683 45.763998043952569 0



guess_batch - duration: 368.40109866999774 seconds
save_work_file - duration: 43.82139843599725 seconds
----- new batch
Processing 20000 guesses more. Total processed so far : 40000/545451
guess_batch - duration: 369.5754677929981 seconds
save_work_file - duration: 43.12204760299937 seconds
----- new batch
Processing 20000 guesses more. Total processed so far : 60000/545451


...

guess_batch - duration: 360.03212274900216 seconds
save_work_file - duration: 51.618432355997356 seconds
----- new batch
Processing 20000 guesses more. Total processed so far : 300000/545451
guess_batch - duration: 361.97457737399964 seconds
save_work_file - duration: 48.63791289799701 seconds
----- new batch
Processing 20000 guesses more. Total processed so far : 320000/545451
guess_batch - duration: 373.6017839619999 seconds
save_work_file - duration: 47.025249522001104 seconds
----- new batch
Processing 20000 guesses more. Total processed so far : 340000/545451


GEOS_NOTICE: Self-intersection at or near point 4.8429172124827913 45.754543469095914 0

GEOS_NOTICE: Self-intersection at or near point 4.8429172124827913 45.754543469095914 0



guess_batch - duration: 357.48987095399934 seconds
save_work_file - duration: 46.94734756300022 seconds
----- new batch
Processing 20000 guesses more. Total processed so far : 360000/545451


GEOS_NOTICE: Self-intersection at or near point 4.8322769249061324 45.734225712924491 0

GEOS_NOTICE: Self-intersection at or near point 4.8322769249061324 45.734225712924491 0

GEOS_NOTICE: Ring Self-intersection at or near point 4.8574109999999999 45.745554040000002 0

GEOS_NOTICE: Ring Self-intersection at or near point 4.8574109999999999 45.745554040000002 0



guess_batch - duration: 358.83932441399884 seconds
save_work_file - duration: 47.52925393699843 seconds
----- new batch
Processing 20000 guesses more. Total processed so far : 380000/545451


In [ ]:
# Report

guesser = Guesser()
guesser.load_work_file(workfile_path)

guesser.report()